In [92]:
import pandas
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation
from sklearn.neural_network import BernoulliRBM
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [5]:
train = pandas.read_csv("train.csv")
test = pandas.read_csv("test.csv")

#Preprocessing
We first examine the shape of training set and test set. There are 42000 training samples 
and 28000 test samples, each with 784 features/pixels. Each featuere represents the intensity of the pixels with an integer between 0 to 255, inclusive. The first column of the training set gives the label of the number.

In [18]:
print(train.shape)
print(test.shape)
print(train.describe())

(42000, 785)
(28000, 784)
              label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  \
count  42000.000000   42000   42000   42000   42000   42000   42000   42000   
mean       4.456643       0       0       0       0       0       0       0   
std        2.887730       0       0       0       0       0       0       0   
min        0.000000       0       0       0       0       0       0       0   
25%        2.000000       0       0       0       0       0       0       0   
50%        4.000000       0       0       0       0       0       0       0   
75%        7.000000       0       0       0       0       0       0       0   
max        9.000000       0       0       0       0       0       0       0   

       pixel7  pixel8    ...         pixel774      pixel775      pixel776  \
count   42000   42000    ...     42000.000000  42000.000000  42000.000000   
mean        0       0    ...         0.219286      0.117095      0.059024   
std         0       0    ...   

Not much preprocessing is needed for this dataset, since there are no missing values and everything is very well formatted.
# Prediction
Let's first attempt to evalute this dataset using random forest.

In [51]:
feature_names = list(train.columns.values[1:])
alg_rf = RandomForestClassifier(random_state = 1, 
                                n_estimators = 100, 
                                min_samples_split = 4, 
                                min_samples_leaf = 2)
score = cross_validation.cross_val_score(alg_rf, train[feature_names], train["label"], cv = 3)
print(score, score.mean())

[ 0.96008283  0.95970853  0.96313759] 0.960976318117


After tweaking the parameters of the RF classifier, the best score we can come up is 0.96097.
Let's try Neural Network model. The following choice of hyperparameter seems to be good for a small sample size of 5000. A more rigirous way to find the correct combinations would be to use GridSearchCV. For now we will use these choice for our model.

In [148]:
%%capture
logistic = LogisticRegression()
logistic.C = 1.0
rbm = BernoulliRBM(random_state = 1, verbose = True)
rbm.learning_rate = 0.02
rbm.n_components = 256
rbm.batch_size = 20
rbm.n_iter = 5
alg_nn = Pipeline(steps = [('rbm', rbm), ('logistic', logistic)])
train_s = train[feature_names]/255.0

train_label = train["label"]
test_N = 1000

score = cross_validation.cross_val_score(alg_nn, train_s.iloc[1:test_N,:], train_label.iloc[1:test_N] , cv = 3);
print(score, score.mean())

Read more about the hyperparameters that NN uses [here](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html#sklearn.neural_network.BernoulliRBM)
and logistic regression [here](http://scikit-learn.org/stable/modules/generated/sklearn.neural_network.BernoulliRBM.html#sklearn.neural_network.BernoulliRBM.). Hyper-parameters can be searched using GridSearchCV.

In [77]:
alg_rf.fit(train_s, train_label)
prediction_rf = alg_rf.predict(test[feature_names]/255.0)


In [110]:
prediction_rf_prob = alg_rf.predict_proba(test[feature_names]/255.0)
print( prediction_rf_prob[1,:] )

[ 0.955  0.     0.005  0.     0.01   0.03   0.     0.     0.     0.   ]


In [133]:
rbm.learning_rate = 0.02
rbm.n_components = 256
rbm.batch_size = 20
rbm.n_iter = 100

alg_nn.fit(train_s, train_label)

[BernoulliRBM] Iteration 1, pseudo-likelihood = -103.13, time = 29.67s
[BernoulliRBM] Iteration 2, pseudo-likelihood = -89.86, time = 36.32s
[BernoulliRBM] Iteration 3, pseudo-likelihood = -82.41, time = 35.78s
[BernoulliRBM] Iteration 4, pseudo-likelihood = -78.01, time = 35.99s
[BernoulliRBM] Iteration 5, pseudo-likelihood = -75.19, time = 35.57s
[BernoulliRBM] Iteration 6, pseudo-likelihood = -74.67, time = 35.52s
[BernoulliRBM] Iteration 7, pseudo-likelihood = -72.05, time = 35.52s
[BernoulliRBM] Iteration 8, pseudo-likelihood = -71.70, time = 35.60s
[BernoulliRBM] Iteration 9, pseudo-likelihood = -69.97, time = 35.58s
[BernoulliRBM] Iteration 10, pseudo-likelihood = -70.45, time = 35.51s
[BernoulliRBM] Iteration 11, pseudo-likelihood = -69.10, time = 36.77s
[BernoulliRBM] Iteration 12, pseudo-likelihood = -68.26, time = 35.70s
[BernoulliRBM] Iteration 13, pseudo-likelihood = -68.83, time = 35.43s
[BernoulliRBM] Iteration 14, pseudo-likelihood = -68.01, time = 35.51s
[BernoulliRBM]

Pipeline(steps=[('rbm', BernoulliRBM(batch_size=20, learning_rate=0.02, n_components=256, n_iter=100,
       random_state=1, verbose=True)), ('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0))])

In [109]:
print(  prediction_rf_prob.argmax(axis = 1)  ) 

[2 0 9 ..., 3 9 2]


In [134]:
prediction_nn = alg_nn.predict_proba(test[feature_names]/255.0)
print(  prediction_nn.argmax(axis = 1)  )  

[2 0 9 ..., 3 9 2]


In [137]:
prediction_ensemble = prediction_nn + prediction_rf_prob
print(prediction_ensemble.argmax(axis = 1))


[2 0 9 ..., 3 9 2]


#Summary
The best score obtained so far is by adding the probability prediction of random forest and neural network, and choosing the label with the maximum probability. The final submission is ensemble of these two methods, which gave a score of 96.8% accuracy. The best method to date uses convolutional neural network and can reach up to 99.8% accuracy.

In [138]:
submission = pandas.DataFrame({
        "ImageId" : range(1,len(test)+1),
        "Label": prediction_ensemble.argmax(axis = 1),
    })
print(submission.head(10))

   ImageId  Label
0        1      2
1        2      0
2        3      9
3        4      9
4        5      3
5        6      7
6        7      0
7        8      3
8        9      0
9       10      3


In [139]:
submission.to_csv("kaggle3.csv", index = False)